In [1]:
# Imports
# Basic
import pandas as pd
import numpy as np
from tqdm import tqdm
import string
import json
import math
import difflib
# Visulaization
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
# Import Libraries
#Basic
from scipy.spatial.distance import cosine
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
#Dimensionality Reduction
import umap
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
#Clustering
import hdbscan
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture

In [36]:
!ls

GLDS-120_rna_seq_Normalized_Counts.csv
GLDS-120_rna_seq_differential_expression.csv
Shulse_celltype_cluster_top50.csv
Shulse_celltype_cluster_top50.xlsx
Shulse_ontologies.xlsx
add_var.ipynb
shahan_celltype_devstage.csv
shahan_celltype_devstage.xlsx
shahan_cluster_labels.tsv


In [146]:
# 24501 genes
glds120=pd.read_csv('GLDS-120_rna_seq_Normalized_Counts.csv')

In [147]:
glds120

,gene,Atha_Col-0-PhyD_root_FLT_Alight_Rep1_GSM2493783,Atha_Col-0-PhyD_root_FLT_Alight_Rep2_GSM2493784,Atha_Col-0-PhyD_root_FLT_Alight_Rep3_GSM2493785,Atha_Col-0-PhyD_root_FLT_dark_Rep1_GSM2493792,Atha_Col-0-PhyD_root_FLT_dark_Rep2_GSM2493793,Atha_Col-0-PhyD_root_FLT_dark_Rep3_GSM2493794,Atha_Col-0-PhyD_root_GC_Alight_Rep1_GSM2493765,Atha_Col-0-PhyD_root_GC_Alight_Rep2_GSM2493766,Atha_Col-0-PhyD_root_GC_Alight_Rep3_GSM2493767,...,Atha_Ws_root_FLT_Alight_Rep3_GSM2493782,Atha_Ws_root_FLT_dark_Rep1_GSM2493789,Atha_Ws_root_FLT_dark_Rep2_GSM2493790,Atha_Ws_root_FLT_dark_Rep3_GSM2493791,Atha_Ws_root_GC_Alight_Rep1_GSM2493762,Atha_Ws_root_GC_Alight_Rep2_GSM2493763,Atha_Ws_root_GC_Alight_Rep3_GSM2493764,Atha_Ws_root_GC_dark_Rep1_GSM2493771,Atha_Ws_root_GC_dark_Rep2_GSM2493772,Atha_Ws_root_GC_dark_Rep3_GSM2493773
0,AT1G01010,236.894398,251.854441,447.882093,355.866612,83.014031,80.314282,209.462831,384.099052,195.656413,...,174.102769,103.071096,277.414530,291.679250,108.408201,239.989562,183.525028,57.513977,144.150163,181.895110
1,AT1G01020,599.339310,562.535087,548.717065,528.053239,672.846432,614.593616,671.398500,490.402086,661.699942,...,483.389017,452.485822,514.546675,410.638611,584.467897,401.248970,471.044609,435.280565,456.300387,467.235795
2,AT1G01030,41.644516,58.224540,76.508502,85.349206,167.929001,231.446405,89.890226,179.360596,135.655571,...,78.773304,40.293634,98.473676,131.440776,62.115645,96.211753,78.234077,89.272772,47.444358,76.053437
3,AT1G01040,140.882999,127.227743,132.690900,146.177937,139.693875,169.513652,154.649974,182.543380,115.512512,...,214.206977,220.600581,192.645428,310.213934,214.748875,121.087622,166.078598,183.869989,260.059509,276.556394
4,AT1G01046,1.407743,0.000000,0.000000,0.988308,0.000000,0.000000,0.000000,2.479491,0.000000,...,1.075047,0.000000,0.000000,0.000000,0.000000,0.000000,5.038375,0.799218,1.245687,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24496,ATMG01370,37.789909,34.381042,13.162677,22.988510,46.090634,31.102641,37.159951,16.347227,24.785574,...,29.407444,35.512840,26.541032,58.523625,30.625102,19.670479,18.167203,28.460959,43.611835,72.780807
24497,ATMG01380,17.963817,31.856922,17.134029,32.867467,56.057324,52.730662,18.384250,30.541042,27.228656,...,22.774987,22.587389,46.840692,83.360765,33.886275,11.953978,33.640926,40.123943,29.855722,67.551486
24498,ATMG01390,11231.668640,9200.979708,11508.551570,13547.818780,16345.608880,19216.452670,13048.964480,18999.112900,15280.827450,...,11775.035650,9959.941265,13711.090580,18680.420130,13654.419450,8659.097306,10103.288980,12744.006240,12113.379120,23549.290790
24499,ATMG01400,1.448979,0.000000,1.011549,0.000000,0.000000,3.226876,0.000000,0.000000,0.990486,...,0.000000,2.533072,3.316439,4.403361,2.884479,0.983258,2.304321,0.811780,11.509522,8.206527


In [28]:
# 1200 with 100 genes each in 12 clusters #875 unique genes
sc_clus=pd.read_csv('shahan_cluster_labels.tsv', sep='\t')

In [29]:
# 32 celltype-devstage with # 16508 entries(7867 unique genes)
sct=pd.read_csv('shahan_celltype_devstage.csv')

In [230]:
# Total of 8028 will have additional information from Shahan et al. paper
len(set(sc_clus['genes']))-len(pd.Series(list(set(sc_clus['genes']).intersection(set(sct['gene'])))))+ len(set(sct['gene']))
    

8028

In [148]:
# Add cluster label from Shahan paper
glds120['shahan_author_label'] = 'No label'
glds120['shahan_cell_type'] = 'Not assigned'

In [224]:
# Example of gene with more than 1 cell type
sct.loc[sct['gene']=='AT4G11210']

,myAUC,avg_diff,power,pct.1,pct.2,cluster,gene,n,Name,cluster_gene,pct.diff,pct.diff_rank,avg_diff_rank,myAUC_rank,combined_rank,dev stage,celltype,plot
450,0.902,1.895652,0.804,0.922,0.190,Elongating-Procambium,AT4G11210,5,DIR14,Elongating-Procambium_AT5G23450,0.732,28,193,39,66,Elongating,Procambium,NaN
789,0.886,1.721157,0.772,0.893,0.214,Meristematic-Procambium,AT4G11210,5,DIR14,Meristematic-Procambium_AT3G09700,0.679,78,87,100,44,Meristematic,Procambium,NaN
3335,0.785,1.392469,0.570,0.676,0.230,Meristematic-Xylem,AT4G11210,5,DIR14,Meristematic-Xylem_AT3G18230,0.446,691,515,579,651,Meristematic,Xylem,NaN
4220,0.825,1.441644,0.650,0.686,0.215,Elongating-Phloem,AT4G11210,5,DIR14,Elongating-Phloem_AT1G44740,0.471,266,286,155,242,Elongating,Phloem,NaN
5417,0.901,1.816133,0.802,0.915,0.191,Elongating-Pericycle,AT4G11210,5,DIR14,Elongating-Pericycle_AT1G10990,0.724,14,101,20,32,Elongating,Pericycle,NaN


In [156]:
# If a gene has more than 1 cell_type, orderby myAUC
# Gene with 1 cell type assigned: 3322 genes, 2 cell type: 2211 genes
# 3 cell type: 1236 genes, 4 or more: 1088 genes
for i in tqdm(set(sct['gene'])):
    cell_type=', '.join(list(sct.loc[sct['gene']==i].sort_values(by=['myAUC'], ascending=False)['cluster']))
    if len(glds120.loc[glds120['gene']==i]) != 0:
        ind=glds120.loc[glds120['gene']==i].index[0]
        glds120['shahan_cell_type'].iloc[ind]=cell_type
    else:
        continue

  0%|                                                  | 0/7867 [00:00<?, ?it/s]/var/folders/6j/lkz2v7js2vq494yrtl79cs0h0000gn/T/ipykernel_66408/231345968.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  glds120['shahan_cell_type'].iloc[ind]=cell_type
100%|██████████████████████████████████████| 7867/7867 [00:21<00:00, 370.96it/s]


In [162]:
# 16668 out of 24501 genes in glds120 are not assigned
# 7833 assigned, out of 7867 genes with cell types
# 34 genes from Shahan et al. dataset were not included in glds120 dataset
len(glds120.loc[glds120['shahan_cell_type']=='Not assigned'])

16668

In [223]:
# Example with more than 1 cluster labels
sc_clus.loc[sc_clus['genes']=='AT4G09030']

,cluster,ref,rank,genes,scores,logfoldchanges,pvals,pvals_adj
383,pericycle cell,rest,83,AT4G09030,81.46152,4.680738,0.0,0.0
427,phloem,rest,27,AT4G09030,69.96600,5.111056,0.0,0.0
532,procambial cell,rest,32,AT4G09030,89.24486,4.983790,0.0,0.0
842,root endodermis,rest,42,AT4G09030,88.43170,5.174338,0.0,0.0


In [231]:
# If a gene has more than 1 cluster label, orderby rank
# Gene with 1 cluster assigned: 619 genes, 2 labels: 203 genes
# 3 labels: 42 genes, 4 or more: 11 genes
for i in tqdm(set(sc_clus['genes'])):
    labels=', '.join(list(sc_clus.loc[sc_clus['genes']==i].sort_values(by=['rank'])['cluster']))
    if len(glds120.loc[glds120['gene']==i]) != 0:
        ind=glds120.loc[glds120['gene']==i].index[0]
        glds120['shahan_author_label'].iloc[ind]=labels
    else:
        continue

  0%|                                                   | 0/875 [00:00<?, ?it/s]/var/folders/6j/lkz2v7js2vq494yrtl79cs0h0000gn/T/ipykernel_66408/1177016527.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  glds120['shahan_author_label'].iloc[ind]=labels
100%|████████████████████████████████████████| 875/875 [00:01<00:00, 450.96it/s]


In [236]:
# 23626 out of 24501 genes in glds120 are not assigned # All genes were included
len(glds120.loc[glds120['shahan_author_label']=='No label'])

23626

In [37]:
 # 497 unique genes # 17 clusters # 339 genes already in sct
ctc=pd.read_csv('Shulse_celltype_cluster_top50.csv')

In [322]:
# Add cluster label from Shulse paper
glds120['shulse_label'] = 'No label'

In [323]:
# Example with more than 1 cluster labels
ctc.loc[ctc['Gene ID']=='AT3G60245']

,Gene ID,Gene Symbol,p-value,Average Difference,Fraction Expressing w/in Cluster,Fraction Expressing Outside Cluster,Cluster
56,AT3G60245,NaN,1.310000e-160,1.52,0.89,0.32,Non-hair cells II
122,AT3G60245,NaN,0.000000e+00,2.02,0.94,0.30,Non-hair cells IV
183,AT3G60245,NaN,1.380000e-47,0.76,0.83,0.33,Columella I
637,AT3G60245,NaN,7.040000e-63,1.70,0.92,0.33,Stele III


In [324]:
# If a gene has more than 1 cluster label, orderby p-value
# Gene with 1 cluster assigned: 330 genes, 2 labels: 124 genes
# 3 labels: 32 genes, 4 or more: 11 genes
for i in tqdm(set(ctc['Gene ID'])):
    labels=', '.join(list(ctc.loc[ctc['Gene ID']==i].sort_values(by=['p-value'])['Cluster']))
    if len(glds120.loc[glds120['gene']==i]) != 0:
        ind=glds120.loc[glds120['gene']==i].index[0]
        glds120['shulse_label'].iloc[ind]=labels
    else:
        continue

  0%|                                                   | 0/497 [00:00<?, ?it/s]/var/folders/6j/lkz2v7js2vq494yrtl79cs0h0000gn/T/ipykernel_66408/3804799620.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  glds120['shulse_label'].iloc[ind]=labels
100%|████████████████████████████████████████| 497/497 [00:01<00:00, 465.88it/s]


In [327]:
# 24004 out of 24501 genes in glds120 are not assigned # All genes were included
len(glds120.loc[glds120['shulse_label']=='No label'])

24004

In [335]:
# Example
glds120.iloc[:,-3:].sample(n=3)

,shahan_author_label,shahan_cell_type,shulse_label
1529,No label,"Meristematic-Endodermis, Meristematic-Cortex",No label
8613,No label,Not assigned,No label
13226,root endodermis,Not assigned,"Endodermis I, Endodermis II"


In [336]:
pd.DataFrame(glds120).to_csv('glds120_nc_singlecell.csv', encoding='utf-8', index=False)